In [9]:
import docker
import subprocess

client = docker.from_env()
running_containers = client.containers.list()
sharedir = "share" 

for cont in running_containers:
    if "omero" and "server" in cont.name:
        print(cont.name)
        ome_server = cont.name
        break
        
subprocess.call(["docker", "exec", ome_server, "/opt/omero/server/OMERO.server/bin/omero", "-s" "localhost", "-u", "root", "login", "-w", "omero"])
test = subprocess.check_output(["docker", "exec", ome_server, "/opt/omero/server/OMERO.server/bin/omero", "obj", "new", "Dataset", "name=annotation_dummies"])
test = test.decode().strip()
subprocess.call(["docker", "exec", ome_server, "/opt/omero/server/OMERO.server/bin/omero", "import", "-d", test, "--transfer=ln_s", "/OMERO/{}".format(sharedir)])

ome-test-omero-server


Using session for root@localhost:4064. Idle timeout: 10 min. Current group: system
Using session for root@localhost:4064. Idle timeout: 10 min. Current group: system
2021-09-28 07:51:34,748 385        [      main] INFO          ome.formats.importer.ImportConfig - OMERO.blitz Version: 5.5.8
2021-09-28 07:51:34,766 403        [      main] INFO          ome.formats.importer.ImportConfig - Bioformats version: 6.5.1 revision: 6f50e4d52c9d96112635fd8b2dde737f31041cf0 date: 7 July 2020
2021-09-28 07:51:34,816 453        [      main] INFO   formats.importer.cli.CommandLineImporter - Setting transfer to ln_s
2021-09-28 07:51:34,819 456        [      main] INFO   formats.importer.cli.CommandLineImporter - Log levels -- Bio-Formats: ERROR OMERO.importer: INFO
2021-09-28 07:51:35,167 804        [      main] INFO      ome.formats.importer.ImportCandidates - Depth: 4 Metadata Level: MINIMUM
2021-09-28 07:51:35,817 1454       [      main] INFO      ome.formats.importer.ImportCandidates - 5 file(s) pa

Image:31,32,33


2021-09-28 07:51:44,368 10005      [2-thread-1] INFO   ormats.importer.cli.LoggingImportMonitor - FILESET_UPLOAD_START
2021-09-28 07:51:44,372 10009      [3-thread-1] INFO   s.importer.transfers.SymlinkFileTransfer - Transferring /OMERO/share/SS7534_1_2_1062344.svs...
2021-09-28 07:51:44,437 10074      [3-thread-1] INFO   ormats.importer.cli.LoggingImportMonitor - FILE_UPLOAD_STARTED: /OMERO/share/SS7534_1_2_1062344.svs
2021-09-28 07:51:45,849 11486      [3-thread-1] INFO   ormats.importer.cli.LoggingImportMonitor - FILE_UPLOAD_COMPLETE: /OMERO/share/SS7534_1_2_1062344.svs
2021-09-28 07:51:45,888 11525      [2-thread-1] INFO   ormats.importer.cli.LoggingImportMonitor - FILESET_UPLOAD_END
2021-09-28 07:51:45,906 11543      [2-thread-1] INFO   ormats.importer.cli.LoggingImportMonitor - IMPORT_STARTED Logfile: 116
2021-09-28 07:51:46,192 11829      [l.Client-1] INFO   ormats.importer.cli.LoggingImportMonitor - METADATA_IMPORTED Step: 1 of 5  Logfile: 116
2021-09-28 07:51:47,149 12786     

Image:34,35,36


...

2021-09-28 07:51:54,726 20363      [2-thread-1] INFO   ormats.importer.cli.LoggingImportMonitor - FILESET_UPLOAD_START
2021-09-28 07:51:54,729 20366      [3-thread-1] INFO   s.importer.transfers.SymlinkFileTransfer - Transferring /OMERO/share/SS7534_1_4_1062346.svs...
2021-09-28 07:51:54,787 20424      [3-thread-1] INFO   ormats.importer.cli.LoggingImportMonitor - FILE_UPLOAD_STARTED: /OMERO/share/SS7534_1_4_1062346.svs
2021-09-28 07:52:06,930 32567      [3-thread-1] INFO   ormats.importer.cli.LoggingImportMonitor - FILE_UPLOAD_COMPLETE: /OMERO/share/SS7534_1_4_1062346.svs
2021-09-28 07:52:06,987 32624      [2-thread-1] INFO   ormats.importer.cli.LoggingImportMonitor - FILESET_UPLOAD_END
2021-09-28 07:52:07,003 32640      [2-thread-1] INFO   ormats.importer.cli.LoggingImportMonitor - IMPORT_STARTED Logfile: 122
2021-09-28 07:52:07,285 32922      [l.Client-0] INFO   ormats.importer.cli.LoggingImportMonitor - METADATA_IMPORTED Step: 1 of 5  Logfile: 122
2021-09-28 07:52:08,350 33987     

Image:40,41,42


2021-09-28 07:52:09,296 34933      [2-thread-1] INFO   ormats.importer.cli.LoggingImportMonitor - FILESET_UPLOAD_START
2021-09-28 07:52:09,299 34936      [3-thread-1] INFO   s.importer.transfers.SymlinkFileTransfer - Transferring /OMERO/share/SS7534_1_3_1062345.svs...
2021-09-28 07:52:09,355 34992      [3-thread-1] INFO   ormats.importer.cli.LoggingImportMonitor - FILE_UPLOAD_STARTED: /OMERO/share/SS7534_1_3_1062345.svs
2021-09-28 07:52:17,395 43032      [3-thread-1] INFO   ormats.importer.cli.LoggingImportMonitor - FILE_UPLOAD_COMPLETE: /OMERO/share/SS7534_1_3_1062345.svs
2021-09-28 07:52:17,456 43093      [2-thread-1] INFO   ormats.importer.cli.LoggingImportMonitor - FILESET_UPLOAD_END
2021-09-28 07:52:17,476 43113      [2-thread-1] INFO   ormats.importer.cli.LoggingImportMonitor - IMPORT_STARTED Logfile: 125
2021-09-28 07:52:17,763 43400      [l.Client-1] INFO   ormats.importer.cli.LoggingImportMonitor - METADATA_IMPORTED Step: 1 of 5  Logfile: 125
2021-09-28 07:52:18,811 44448     

Image:43,44,45


0

In [11]:
from omero.gateway import BlitzGateway, MapAnnotationWrapper
from omero.model import EllipseI

class Ome_Annot_extractor():
    """
        Connection Manager and Methods for extracting annotations from an Omero Host
        :param ome_server: Host name
        :param user: User
        :param pw: Password
    """
    
    def __init__(self, user, pw, ome_server):
        self.user = user
        self.pw = pw
        self.server = ome_server
        self.conn = None
    
    
    def connect(self):
        """
            Connect to an OMERO server
        """
        self.conn = BlitzGateway(self.user, self.pw,
                            host=self.server, secure=False)
        self.conn.connect()
        self.conn.c.enableKeepAlive(60)
    return conn


    def disconnect(self):
        """
            Disconnect from an OMERO server
            :param conn: The BlitzGateway
        """
        assert self.conn is not None, "Connection not established please use "
        self.conn.close()
    
    def list_projects(self):
        """
            List the projects available to the current user
        """
        my_exp_id = self.conn.getUser().getId()
        default_group_id = self.conn.getEventContext().groupId
        projects =  conn.getObjects("Project", opts={'owner': my_exp_id,
                                            'group': default_group_id,
                                            'order_by': 'lower(obj.name)',
                                            'limit': 5, 'offset': 0})
        for project in projects:
            print_obj(project)
        
        return projects
    
    def select_project(self, sel_project):
        projects = self.list_projects()
        
        
    
    def list_datasets(self, project):
        """
            List the projects available to the current user
            :param projects: List of projects
        """
        pass
    
    def list_images(self):
        pass
        


ImportError: /opt/conda/lib/python3.9/site-packages/IcePy.cpython-39-x86_64-linux-gnu.so: undefined symbol: libiconv_open

In [10]:
ome_parse = Ome_Annot_Extractor("root", "omero", ome_server)

NameError: name 'Ome_Annot_Extractor' is not defined